# VACATIONPY

#### Starter Code to Import Libraries and Load the Weather and Coordinates

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgetown,5.4112,100.3354,26.96,89,20,2.06,MY,2024-04-08 15:35:50
1,1,chandbali,20.7833,86.7667,29.73,33,100,2.70,IN,2024-04-08 15:36:37
2,2,anadyr,64.7500,177.4833,-17.72,78,0,1.00,RU,2024-04-08 15:33:23
3,3,atafu village,-8.5421,-172.5159,29.47,75,16,4.41,TK,2024-04-08 15:36:37
4,4,rio grande,-32.0350,-52.0986,24.06,74,39,5.18,BR,2024-04-08 15:36:33


---

#### 1. Map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
mat_plot = city_data_df.hvplot.points("Lng","Lat",geo=True, size="Humidity", scale = 1, color = "City", alpha = 0.5, tiles = "OSM", frame_wight = 700, frame_height = 500)

mat_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

---

#### 2. Narrow down `city_data_df` DataFrame to find the ideal weather condititons

In [4]:
#Narrow down out DataFrame and find the ideal weather conditions. 
#Max temperature should be lower then 27 degrees but higher than 21 degree
#wind speed should be less then 4.5 m/s
#zero cloudiness

ideal_weather_conditions = city_data_df.loc[(city_data_df['Max Temp'] < 27)&(city_data_df['Max Temp'] > 21)
                                             & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

ideal_weather_conditions

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,fort bragg,35.1390,-79.0060,22.12,50,0,4.12,US,2024-04-08 15:35:57
74,74,callao,-12.0667,-77.1500,23.16,83,0,3.09,PE,2024-04-08 15:36:33
127,127,diego de almagro,-26.3667,-70.0500,26.22,24,0,2.88,CL,2024-04-08 15:36:52
166,166,vicam,27.6333,-110.2833,21.63,29,0,2.31,MX,2024-04-08 15:36:56
227,227,kailua-kona,19.6406,-155.9956,22.98,80,0,1.54,US,2024-04-08 15:36:33
298,298,mount isa,-20.7333,139.5000,21.87,40,0,4.12,AU,2024-04-08 15:37:12
381,381,kone,-21.0595,164.8658,22.10,95,0,2.00,NC,2024-04-08 15:37:22
394,394,vallenar,-28.5708,-70.7581,24.44,46,0,2.23,CL,2024-04-08 15:37:23
409,409,west plains,36.7281,-91.8524,22.34,34,0,3.60,US,2024-04-08 15:37:25
416,416,umluj,25.0213,37.2685,24.70,45,0,2.53,SA,2024-04-08 15:37:26


In [5]:
len(ideal_weather_conditions)

10

---

#### 3. Create a new DataFrame that will store city,country, coordinates and humidity

In [6]:
hotel_df = ideal_weather_conditions[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,fort bragg,US,35.1390,-79.0060,50,
74,callao,PE,-12.0667,-77.1500,83,
127,diego de almagro,CL,-26.3667,-70.0500,24,
166,vicam,MX,27.6333,-110.2833,29,
227,kailua-kona,US,19.6406,-155.9956,80,
298,mount isa,AU,-20.7333,139.5000,40,
381,kone,NC,-21.0595,164.8658,95,
394,vallenar,CL,-28.5708,-70.7581,46,
409,west plains,US,36.7281,-91.8524,34,
416,umluj,SA,25.0213,37.2685,45,


---

#### 4. For each city use Geoapify API to find the first hotel locateed within 10,000 metres of your coordinates 

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(requests.get(base_url, params=params))
    
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
callao - nearest hotel: Casa Ronald
diego de almagro - nearest hotel: Residencial Hambyll
vicam - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
mount isa - nearest hotel: Ibis Styles
kone - nearest hotel: Pacifik Appartels
vallenar - nearest hotel: Hotel del Marqués
west plains - nearest hotel: Hampton
umluj - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
27,fort bragg,US,35.1390,-79.0060,50,Airborne Inn Lodging
74,callao,PE,-12.0667,-77.1500,83,Casa Ronald
127,diego de almagro,CL,-26.3667,-70.0500,24,Residencial Hambyll
166,vicam,MX,27.6333,-110.2833,29,No hotel found
227,kailua-kona,US,19.6406,-155.9956,80,Kona Seaside Hotel
298,mount isa,AU,-20.7333,139.5000,40,Ibis Styles
381,kone,NC,-21.0595,164.8658,95,Pacifik Appartels
394,vallenar,CL,-28.5708,-70.7581,46,Hotel del Marqués
409,west plains,US,36.7281,-91.8524,34,Hampton
416,umluj,SA,25.0213,37.2685,45,No hotel found


---

#### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
mat_plot = hotel_df.hvplot.points("Lng","Lat",geo=True, scale = 1, color = "City", alpha = 0.5, tiles = "OSM", frame_wight = 700, frame_height = 500, hover_cols=["Hotel Name","City","Country"])

mat_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)